In [ ]:
%matplotlib widget
%matplotlib widget

import ipywidgets as widgets
from IPython.display import clear_output

import jupyros as jr
import rospy

from std_msgs.msg import String
from final_assignment.srv import Directions

from IPython.display import display

from geometry_msgs.msg import Twist, Vector3 

from jupyros import ros3d
import os

from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from ipywidgets import interact, interactive
import matplotlib.pyplot as plt
import tf
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
from std_msgs.msg import Int32

reached_points = []

In [ ]:
def set_velocity(direction):
    
    init = Vector3(0, 0, 0)
    repost = Twist( init, init)
    
    #different velocities for different input
    if direction =='w':
        repost.linear.x = velocity.value
    
    elif direction == 's':
        repost.linear.x = -velocity.value
    
    elif direction == 'd':
        repost.angular.z = -velocity.value
    
    elif direction == 'a':
        repost.angular.z = velocity.value
    
    #initialize the publisher to pubblic on topic remap_cmd_vel 
    pub = rospy.Publisher('remap_cmd_vel',Twist, queue_size=10)
    pub.publish(repost)

In [ ]:
def driving_console():
    #interface to drive the robot with buttons
    display(driving_comm)
    display(check)
    display(bcanc)
    
    up.on_click(on_button_clicked_w)
    down.on_click(on_button_clicked_s)
    left.on_click(on_button_clicked_a)
    right.on_click(on_button_clicked_d)
   
    check.observe(check_handler, names='value')
    bcanc.on_click(on_button_clicked_canc)
    #display the ros3d map
    display_map()

In [ ]:
def menu_choice(choice):
    #callback for events on the modality choice menu
    if choice=='mode1':
        display(coo_x, coo_y, bstart, bcanc)
        bstart.on_click(on_button_clicked_start)
        bcanc.on_click(on_button_clicked_canc)
    else:
        display(velocity)
        driving_console()

In [ ]:
def display_map():
    #display a live map of the working enviroment of the robot
    
    v = ros3d.Viewer()
    rc = ros3d.ROSConnection(url="ws://localhost:9090")
    tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')

    laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
    map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client, continuous=True)
    path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc,
    tf_client=tf_client)
    urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client,
    path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))
    g = ros3d.GridModel()

    v.objects = [laser_view, map_view, urdf]

    display(v)

In [ ]:
#callback functions for the events

def menu_handler(b):
    menu_choice(menu.value)

#back to menu button
def on_button_clicked_canc(b):
    clear_output()

#modify the value of the ROS parameter for obstacle avoidance control
def check_handler(b):
    rospy.set_param("/obstacle_avoidance", check.value)
    
#service call to set the new position to reach       
def on_button_clicked_start(b):
    with output:
        rospy.wait_for_service('directions')
        new_pose = rospy.ServiceProxy('directions', Directions)
        rt = new_pose(coo_x.value , coo_y.value)
        if rt.return_ == 1:
            print("new position reached!")
            reached_points.append('reached')
        else:
            print("time out")
            reached_points.append('not reached')
        display(bcanc)
        bcanc.on_click(on_button_clicked_canc)
         
#handle all the different console buttons and call the function to set the velocity 
def on_button_clicked_w(b):
    set_velocity('w')
def on_button_clicked_s(b):
    set_velocity('s')
def on_button_clicked_a(b):
    set_velocity('a')
def on_button_clicked_d(b):
    set_velocity('d')

In [ ]:
class Visualiser:
    def __init__(self, limit_on_x, fig_title):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'bo')
        self.x_data, self.y_data = [] , []
        self.x_limits=limit_on_x
        self.ax.set_title(fig_title)
        
    def plot_init(self):
        self.ax.set_xlim(-self.x_limits, self.x_limits)
        self.ax.set_ylim(-10, 10)
        return self.ln
    
    def odom_clbk(self, msg):
        self.y_data.append(msg.pose.pose.position.y) #try the append on the scanner
        self.x_data.append(msg.pose.pose.position.x)
       
    def laser_clbk(self, msg):
        self.x_data = msg.ranges
        self.y_data = msg.intensities        
   
    def refresh_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    
class Analysis:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.y_data = [0.05,0.05] 
        self.labels = ['reached', 'failed'] 
        self.bar = self.ax.bar(self.labels, self.y_data, color = 'b', width = 0.3)
        self.ax.set_ylim(top = 10)
        self.ax.set_title('classification of positions reaching')
    
    def objective_clbk(self, msg):
        if msg.data == 1:
            self.y_data[0]+=1
        else :
            self.y_data[1]+=1
            
    def animate(self, frame):
        index = self.y_data[frame]
        self.bar[frame].set_height(index)


rospy.init_node('user_interface')

intro= widgets.Textarea(value='Hello User! please select between the different modalities:',disabled=False)

#create starting menu interface
menu= widgets.RadioButtons(options=['mode1', 'mode2'],value= None, description='driving:', disabled=False)
bstart= widgets.Button(description="start moving")
bcanc = widgets.Button(description="back to menu")

#boxes for entering the coordinate of the taget to reach
coo_x = widgets.FloatText(description="x coordinate: ")
coo_y = widgets.FloatText(description="y coordinate: ")

velocity=widgets.FloatSlider(min=0, max=3, step=0.1, value=1, description='Robot speed:');

#listener for driving choice
menu.observe(menu_handler, names='value')

#console buttons
up = Button(description='UP', layout=Layout(grid_area='b1'))
left = Button(description='LEFT', layout=Layout(grid_area='b2'))
right = Button(description='RIGHT', layout=Layout( grid_area='b3'))
down = Button(description='DOWN', layout=Layout(grid_area='b4'))

HBox([VBox([right, down]),VBox([up, left])])

driving_comm=GridBox(children=[up, left, right, down],layout=Layout(width='50%', grid_template_rows='auto auto',
    grid_template_columns='33% 33% 33% 33%',
    grid_template_areas='''
    " . b1 . "
    "b2 . b3 "
    " . b4 . "
    ''')
)

#checkbox widget for activate/deactivate the collision avoidance option during the manual navigation of the robot
check = widgets.Checkbox(value = False, description='active collision avoidance')

#create an output cell
output = widgets.Output()

#display both button and output cell display main menu of the GUI
display(intro, menu, output)

vis_odom = Visualiser(10, 'Position of the robot')
sub_odom = rospy.Subscriber('/odom', Odometry, vis_odom.odom_clbk)
ani1 = FuncAnimation(vis_odom.fig, vis_odom.refresh_plot, init_func=vis_odom.plot_init)
plt.show(block=True)

vis_scan = Visualiser(100, 'Laser scan measurements')
sub_scan = rospy.Subscriber('/scan', LaserScan, vis_scan.laser_clbk)
ani2 = FuncAnimation(vis_scan.fig, vis_scan.refresh_plot, init_func=vis_scan.plot_init)
plt.show(block=True)

analysis= Analysis()
sub_success = rospy.Subscriber('/goal_reached', Int32, analysis.objective_clbk)
ani3 = FuncAnimation(analysis.fig, analysis.animate, frames=2)
plt.show(block=True)
Textarea(value='Hello User! please select between the different modalities:')
RadioButtons(description='driving:', options=('mode1', 'mode2'), value=None)
Output()
Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …
Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …
Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …
FloatText(value=0.0, description='x coordinate: ')
FloatText(value=0.0, description='y coordinate: ')
Button(description='start moving', style=ButtonStyle())
Button(description='back to menu', style=ButtonStyle())